6.7 Exercise Problem2
---
2232036001 권준형

물리학에서 세 번째로 어려운 문제인, 새로운 발견 성과를 자축하기 위한 적당한 와인을 찾는 문제를 해결하기 위한 적당한 수의 입력 파라미터를 갖는 모델을 만들어라.

In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, precision=2, linewidth=75)

 Numpy를 사용해 와인 정보(4챕터, 4.3 테이블 데이터 표현하기 참고)를 가져오고, Numpy 배열을 파이토치 텐서로 변환하는 작업을 수행한다.

In [4]:
import csv
wine_path = "C:/Users/jhkwo/DataScience/data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
                         skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

CSV파일로부터 와인 정보를 가져와 float32 타입의 2차원 배열을 생성했다.

테이블 데이터의 각 열의 타이틀을 확인한다. 

In [5]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

넘파이 배열을 텐서로 변환한다.

In [6]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

데이터와 점수를 구분한다.

In [8]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [9]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [10]:
target = wineq[:, -1].long()
target

tensor([6, 6,  ..., 7, 6])

6.1 인공 뉴런 파트를 참고하여, 데이터의 일부는 훈련용으로, 일부는 평가용으로 나눈다.
평가용 데이터는 20%로 한다.

In [35]:
n_samples= data.shape[0]

n_val = int(0.2 * n_samples)

suffled_indices = torch.randperm(n_samples)

train_indices = suffled_indices[:-n_val]
val_indices = suffled_indices[-n_val:]

In [44]:
x_train = data[train_indices,:]
y_train = target[train_indices]

x_val = data[val_indices,:]
y_val = target[val_indices]

온도 문제에서는 1차원의 텐서를 입력데이터로 활용하였다.(t_u, t_c 쌍)

와인 문제는 11개의 평가지표로부터 점수를 구하는 문제로, 입력 사이즈를 11로 설정한다.

우선, 은닉층의 파라미터는 교재에서와 같이 13을 사용하여 학습을 시켜본다.

In [45]:
import torch.optim as optim
import torch.nn as nn

seq_model = nn.Sequential(
    nn.Linear(11, 13),
    nn.Tanh(),
    nn.Linear(13,1))
seq_model

Sequential(
  (0): Linear(in_features=11, out_features=13, bias=True)
  (1): Tanh()
  (2): Linear(in_features=13, out_features=1, bias=True)
)

In [46]:
optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

훈련 루프 정의

In [50]:
def training_loop(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        z_train = model(x_train)
        loss_train = loss_fn(z_train, y_train)

        z_val = model(x_val)
        loss_val = loss_fn(z_val, y_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

time을 사용해서 트레이닝 시간을 측정한다.

In [54]:
xn_train = 0.1 * x_train
xn_val = 0.1 * x_val

In [78]:
import time
start = time.time()

def loss_fn(z, y):
    squared_diffs = (z - y)**2
    return squared_diffs.mean()

seq_model = nn.Sequential(
    nn.Linear(11, 20),
    nn.Tanh(),
    nn.Linear(20,1))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-2)

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    x_train = xn_train,
    x_val = xn_val, 
    y_train = y_train,
    y_val = y_val)

print('output', seq_model(x_val))
print('answer', y_val)

time.sleep(1)

print(f"{time.time() - start:.4f} sec")

Epoch 1, Training loss 34.5676, Validation loss 34.6834
Epoch 1000, Training loss 0.7775, Validation loss 0.8837
Epoch 2000, Training loss 0.7739, Validation loss 0.8716
Epoch 3000, Training loss 0.7724, Validation loss 0.8658
output tensor([[5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87],
        [5.87]

200.7296 sec


모든 데이터를 5.87로 평가한다. 1만회를 하더라도 2천회 이상부터는 손실이 일정 수준 이하로 떨어지지 않으며, 4천회 이상 부터는 손실 값이 거의 고정된다. 모델의 파라미터를 늘려서 학습을 다시 해본다.

a. 온도 데이터보다 시간이 오래 걸린다.
b. 입력 파라미터가 다차원이어서 상관관계를 분석하는 데 시간이 걸리는 게 아닐까 생각된다.
c. 손실이 줄어들기는 하지만 크게 줄어들지 않는 문제가 있다.
d. 데이터셋이 11차원이기 때문에 하나의 그래프로 표현하기는 어렵다. (인간이 이해할 수 없다.)
11개의 그래프를 통해서 표현해야 할 것 같다.